In [27]:
from scipy.io import loadmat
import random

In [5]:
"""
Structure of annotations:

TRAIN: filename, folder, image_size, original_database, person
"""

annotations = loadmat('annotations/Annotations.mat')

In [54]:
print(type(annotations))
print(type(annotations['train']))
print(len(annotations['train']))
print(len(annotations['train'][0]))
print(len(annotations['train'][0][0]))

print("First Sample:")
print(annotations['train'][0][0])

<class 'dict'>
<class 'numpy.ndarray'>
1
17077
5
First Sample:
(array(['COCO_val2014_000000562243.jpg'], dtype='<U29'), array(['mscoco/images'], dtype='<U13'), array([[(array([[640]], dtype=uint16), array([[640]], dtype=uint16))]],
      dtype=[('n_col', 'O'), ('n_row', 'O')]), array([[(array(['mscoco'], dtype='<U6'), array([[(array([[562243]], dtype=int32), array([[448867]], dtype=int32))]],
      dtype=[('image_id', 'O'), ('annotations_id', 'O')]))]],
      dtype=[('name', 'O'), ('info', 'O')]), array([[(array([[ 86,  58, 564, 628]], dtype=uint16), array([[(array([[array(['Disconnection'], dtype='<U13'),
        array(['Doubt/Confusion'], dtype='<U15')]], dtype=object),)]],
      dtype=[('categories', 'O')]), array([[(array([[5]], dtype=uint8), array([[3]], dtype=uint8), array([[9]], dtype=uint8))]],
      dtype=[('valence', 'O'), ('arousal', 'O'), ('dominance', 'O')]), array(['Male'], dtype='<U4'), array(['Adult'], dtype='<U5'))]],
      dtype=[('body_bbox', 'O'), ('annotations_cate

In [215]:
"""
A lot of data is wrapped in single-element arrays (could be
converting from .mat to numpy array). This will make it easier
to work with the data.
"""
train = [{
    'filename'  : sample[0][0],
    'folder'    : sample[1][0],
    'image_size': {
        'width' : sample[2][0][0][0][0][0],
        'height': sample[2][0][0][1][0][0]
    },
    'original_database': sample[3][0][0][0][0],
    'person': [{
        'body_bbox': [i for i in person[0][0]],# can keep this numpy array
        'annotations_categories': [
            category[0] for category in person[1][0][0][0][0]
        ]
    } for person in sample[4][0]]
} for sample in annotations['train'][0]]

print(len(train))
print(random.choice(train))

17077
{'image_size': {'width': 640, 'height': 480}, 'original_database': 'mscoco', 'folder': 'mscoco/images', 'filename': 'COCO_train2014_000000274267.jpg', 'person': [{'body_bbox': [276, 21, 411, 225], 'annotations_categories': ['Anticipation']}]}


In [206]:
train = annotations['train'][0]
for i in range(1):
    sample = random.choice(train)
    #print("filename:   ", sample[0][0])
    #print("folder:     ", sample[1][0])
    #print("image size: ", str(sample[2][0][0][0][0][0]) + " x " + str(sample[2][0][0][1][0][0]))
    #print("len   : ", len(sample[4][0]))
    person = sample[4][0]
    print("person: ", len(person[0][1][0][0][0][0]))
    print("box: ", person[0][0])
    print("cat: ", [category[0] for category in person[0][1][0][0][0][0]])

person:  3
box:  [[208  48 333 425]]
cat:  ['Affection', 'Peace', 'Pleasure']


<class 'numpy.ndarray'>
